# SK Car Sales Descriptions in English and Spanish

- Get a list of cars and for each car get a sales descriptions in English and translate it into Spanish.

## Load the required .NET packages and supporting classes

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230626.1-preview"
#r "nuget: Microsoft.Extensions.Logging.Console, 7.0.0"
#r "nuget: Microsoft.EntityFrameworkCore.Sqlite, 7.0.9"
#r "nuget: dotenv.net, 3.1.2"

using System.ComponentModel;
using System.Net.Http;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.EntityFrameworkCore;
using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#!import Utils/ConsoleLogger.cs
#!import Models/Models.cs

// Json Options
readonly JsonSerializerOptions s_jsonOptions = new() { WriteIndented = true };

In [ ]:
class CarsDbContext : DbContext
{
    public DbSet<CarEF> Users { get; set; }

    public CarsDbContext()
    {
    }

    public CarsDbContext(DbContextOptions<CarsDbContext> options) : base(options)
    {
    }
}

public class CarEF
{
    public string VIN { get; set; }
    public string Make { get; set; }
    public string Model { get; set; }
    public int Year { get; set; }
    public string Package {get;set;}
    public string Motor {get;set;}
    public double Price {get;set;}
}

## Load the OpenAI variables from environment variables or an .env file

In [ ]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_KEY");
Console.WriteLine($"Using deployment: {deploymentName} at: {endpoint} with key {apiKey.Substring(0, 5)}...");

## Mock a database call to get the a list of cars

In [ ]:
var mockCarData = new List<Car>{
    new Car("123","Ford", "Explorer", 2015, "Silver", "V6", "Platinum", 10000, 16500),
    new Car("456","Ford", "Mustang", 2018, "Blue", "V8", "Sports", 10000, 250000),
    new Car("789","Ford", "Escape", 2020, "Red", "V6","Special", 3000, 15000)
  };
  
List<Car> MockDBCall() {  
  return mockCarData;
}

## Function to get a short car description from the car properties

In [ ]:
string GetCarDescription(Car car) {
  return $"{car.Year} {car.Make} {car.Model} {car.Color} {car.Motor} {car.Package} with {car.milage} miles for ${car.price}";
}

## Create an instance of a kernel

In [ ]:
var kernel = new KernelBuilder()
    .WithLogger(ConsoleLogger.Log)
    .WithAzureTextCompletionService(deploymentName,endpoint, apiKey)
    .Build();

## Create the sales definition and translation in-line functions and add them to the kernel

In [ ]:
// Define a function to get a used car sales description
const string fnSalesDefinition = @"Get a used car sales description for the following: 
{{$input}}.";

var salesDescriptionFunc = kernel.CreateSemanticFunction(fnSalesDefinition, 
    maxTokens: 200, temperature: 0.5, topP: 1);

// Define a function to translate English to Spanish
const string fnESTranslatorDefinition = @"Translate the following English text to Spanish: 
{{$input}}.";

var esTranslatorFunc = kernel.CreateSemanticFunction(fnESTranslatorDefinition, 
maxTokens: 200, temperature: 0.5, topP: 1);

## Using the kernel and sk functions, get a car sales description and translate it into Spanish

In [ ]:
// Mock making a call to a database to get a list of cars
var cars = MockDBCall();
var text = kernel.ImportSkill(new TextSkill());
var carDesc = GetCarDescription(cars[0]);

## Get a sales description, translate it and Uppercase it in a pipe

In [ ]:

var pipeLineResult = await kernel.RunAsync(carDesc,
  salesDescriptionFunc,
  esTranslatorFunc,
  text["Uppercase"]);

Console.WriteLine(pipeLineResult);

## Get a sales description and a translation for all the cars in the mock database

In [ ]:
// Run tasks asynchronously
// var task1 = kernel.RunAsync(carDesc,salesDescriptionFunc);
// var task2 = kernel.RunAsync(carDesc,salesDescriptionFunc);
// await Task.WhenAll(task1,task2);
// Console.WriteLine(task1.Result + " " + task2.Result);

var processedCars = new List<CarTranslation>();
foreach(var car in cars) {

  // Get a short description given the car properties
  var desc = GetCarDescription(car);
  Console.WriteLine($"Processing: {desc}");
  
  // Use the SK sales description function to get a sales for a car
  var result = await kernel.RunAsync(desc,salesDescriptionFunc);
  Console.WriteLine($"Sales Description: {result}");

  // Get the usage for the call
  Console.WriteLine(JsonSerializer.Serialize(result.ModelResults.LastOrDefault()?.GetOpenAITextResult()?.Usage, s_jsonOptions));
  
  // Use the SK translation function to translate the sales description into Spanish
  var es = await kernel.RunAsync(result.ToString(), esTranslatorFunc);
  Console.WriteLine($"Translation: {es}");
  
  // Add processed the EN and ES sales description
  processedCars.Add(new CarTranslation(car.VIN, result.ToString(), es.ToString()));
}

// Show the processed cars
processedCars